# Project 3

# Prompt

Dataset(s) to be used:
- NYC MTA Daily Ridership : https://data.ny.gov/Transportation/MTA-Daily-Ridership-Data-2020-2025/vxuj-8kew/about_data
- NYC Central Park Daily Weather : https://www.opendatabay.com/data/ai-ml/f2682956-2163-47b0-8c55-9c4400ac78ae

Analysis question:
Does rainfall significantly reduce NYC subway ridership 2021-2022? If so, how strong is the effect and does it change over time?

Columns that will (likely) be used:
From MTA dataset:
- date
- subways:total_estimated_ridership

From Weather dataset:
- DATE
- PRCP (precipitation in inches)
- TAVG (optional)
- TMAX/TMIN (optional)

Columns used to merge:
- MTA: date
- Weather: DATE

Hypothesis:
Rainy days have lower subway ridership compared to dry days (per unit of rainfall).

In [157]:
import pandas as pd
import plotly.express as px

mta = pd.read_csv("../mta_daily.csv")
weather = pd.read_csv("../central_park_weather.csv")
mta.info()
mta.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1776 entries, 0 to 1775
Data columns (total 15 columns):
 #   Column                                                   Non-Null Count  Dtype 
---  ------                                                   --------------  ----- 
 0   Date                                                     1776 non-null   object
 1   Subways: Total Estimated Ridership                       1776 non-null   object
 2   Subways: % of Comparable Pre-Pandemic Day                1776 non-null   object
 3   Buses: Total Estimated Ridership                         1776 non-null   object
 4   Buses: % of Comparable Pre-Pandemic Day                  1776 non-null   object
 5   LIRR: Total Estimated Ridership                          1776 non-null   object
 6   LIRR: % of Comparable Pre-Pandemic Day                   1776 non-null   object
 7   Metro-North: Total Estimated Ridership                   1776 non-null   object
 8   Metro-North: % of Comparable Pre-Pande

,Date,Subways: Total Estimated Ridership,Subways: % of Comparable Pre-Pandemic Day,Buses: Total Estimated Ridership,Buses: % of Comparable Pre-Pandemic Day,LIRR: Total Estimated Ridership,LIRR: % of Comparable Pre-Pandemic Day,Metro-North: Total Estimated Ridership,Metro-North: % of Comparable Pre-Pandemic Day,Access-A-Ride: Total Scheduled Trips,Access-A-Ride: % of Comparable Pre-Pandemic Day,Bridges and Tunnels: Total Traffic,Bridges and Tunnels: % of Comparable Pre-Pandemic Day,Staten Island Railway: Total Estimated Ridership,Staten Island Railway: % of Comparable Pre-Pandemic Day
0,03/01/2020,"2,212,965",97%,"984,908",99%,"86,790",100%,"55,825",59%,"19,922",113%,"786,960",98%,"1,636",52%
1,03/02/2020,"5,329,915",96%,"2,209,066",99%,"321,569",103%,"180,701",66%,"30,338",102%,"874,619",95%,"17,140",107%
2,03/03/2020,"5,481,103",98%,"2,228,608",99%,"319,727",102%,"190,648",69%,"32,767",110%,"882,175",96%,"17,453",109%
3,03/04/2020,"5,498,809",99%,"2,177,165",97%,"311,662",99%,"192,689",70%,"34,297",115%,"905,558",98%,"17,136",107%
4,03/05/2020,"5,496,453",99%,"2,244,515",100%,"307,597",98%,"194,386",70%,"33,209",112%,"929,298",101%,"17,203",108%


In [158]:
weather.info()
weather.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56245 entries, 0 to 56244
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    56245 non-null  object 
 1   PRCP    56245 non-null  float64
 2   SNOW    56082 non-null  float64
 3   SNWD    39687 non-null  float64
 4   TMIN    56238 non-null  float64
 5   TMAX    56238 non-null  float64
dtypes: float64(5), object(1)
memory usage: 2.6+ MB


,DATE,PRCP,SNOW,SNWD,TMIN,TMAX
0,1869-01-01,0.75,9.0,NaN,19.0,29.0
1,1869-01-02,0.03,0.0,NaN,21.0,27.0
2,1869-01-03,0.00,0.0,NaN,27.0,35.0
3,1869-01-04,0.18,0.0,NaN,34.0,37.0
4,1869-01-05,0.05,0.0,NaN,37.0,43.0


First, I standardized the column names and kept only the columns needed for analysis. Next, I filtered both datasets to include only the years 2021 and 2022, and converted their date formats into a consistent structure. This allows the two datasets to be merged accurately on the date field.

In [159]:
mta.columns = mta.columns.str.lower()
weather.columns = weather.columns.str.lower()

weather['date'] = pd.to_datetime(weather['date'])
mta['date'] = pd.to_datetime(mta['date'], format="%m/%d/%Y")

mta.head()

,date,subways: total estimated ridership,subways: % of comparable pre-pandemic day,buses: total estimated ridership,buses: % of comparable pre-pandemic day,lirr: total estimated ridership,lirr: % of comparable pre-pandemic day,metro-north: total estimated ridership,metro-north: % of comparable pre-pandemic day,access-a-ride: total scheduled trips,access-a-ride: % of comparable pre-pandemic day,bridges and tunnels: total traffic,bridges and tunnels: % of comparable pre-pandemic day,staten island railway: total estimated ridership,staten island railway: % of comparable pre-pandemic day
0,2020-03-01,"2,212,965",97%,"984,908",99%,"86,790",100%,"55,825",59%,"19,922",113%,"786,960",98%,"1,636",52%
1,2020-03-02,"5,329,915",96%,"2,209,066",99%,"321,569",103%,"180,701",66%,"30,338",102%,"874,619",95%,"17,140",107%
2,2020-03-03,"5,481,103",98%,"2,228,608",99%,"319,727",102%,"190,648",69%,"32,767",110%,"882,175",96%,"17,453",109%
3,2020-03-04,"5,498,809",99%,"2,177,165",97%,"311,662",99%,"192,689",70%,"34,297",115%,"905,558",98%,"17,136",107%
4,2020-03-05,"5,496,453",99%,"2,244,515",100%,"307,597",98%,"194,386",70%,"33,209",112%,"929,298",101%,"17,203",108%


In [160]:
weather.head()

,date,prcp,snow,snwd,tmin,tmax
0,1869-01-01,0.75,9.0,NaN,19.0,29.0
1,1869-01-02,0.03,0.0,NaN,21.0,27.0
2,1869-01-03,0.00,0.0,NaN,27.0,35.0
3,1869-01-04,0.18,0.0,NaN,34.0,37.0
4,1869-01-05,0.05,0.0,NaN,37.0,43.0


In [161]:
mta = mta[['date', 'subways: total estimated ridership']]
mta_21_22 = mta[mta['date'].dt.year.between(2021, 2022)]
weather_21_22 = weather[weather['date'].dt.year.between(2021, 2022)]
df = pd.merge(mta_21_22, weather_21_22, on='date', how='inner')
df.head()

,date,subways: total estimated ridership,prcp,snow,snwd,tmin,tmax
0,2021-01-01,"613,692",0.62,0.0,0.0,33.0,40.0
1,2021-01-02,"988,418",0.05,0.0,0.0,36.0,51.0
2,2021-01-03,"653,187",0.22,0.0,0.0,34.0,38.0
3,2021-01-04,"1,557,977",0.02,0.0,0.0,35.0,43.0
4,2021-01-05,"1,597,518",0.00,0.0,0.0,36.0,42.0



Before going deeper, I first plotted ridership vs. precipitation directly.

Spoiler: It’s messy. Rain doesn’t always mean lower ridership — maybe weekdays vs weekends dominate the pattern.

But this exploration shows where the data might mislead us.

In [162]:
fig = px.scatter(
    df,
    x = "prcp",
    y= "subways: total estimated ridership",
    title = "Raw Ridership vs Precipitation"
)
fig.update_layout(xaxis_title_text = "Precipitation (inches)")
fig.update_layout(yaxis_title_text = "Ridership")
fig.show()

fig.write_image("project3_raw_ridership_vs_precipitation.png")




![](project3_raw_ridership_vs_precipitation.png)

Add Day-of-Week

In [163]:
df['dow'] = df['date'].dt.day_name()

Group by PRCP buckets

In [164]:
df['rain_bucket'] = pd.cut(df['prcp'], bins=[0,0.01,0.1,0.5,2], 
                           labels=['No rain','Light','Moderate','Heavy'])

Remove thousands separators and strip leading/trailing spaces, then convert the column to a numeric data type. This ensures the ridership values can be used in mathematical operations such as calculating averages or correlations.

In [165]:

df['subways: total estimated ridership'] = (
    df['subways: total estimated ridership']
    .str.replace(',', '', regex=False)
    .str.strip()
)

df['subways: total estimated ridership'] = pd.to_numeric(
    df['subways: total estimated ridership'],
    errors='coerce'
)

print(df['subways: total estimated ridership'].dtype)
print(df['subways: total estimated ridership'].head())


int64
0     613692
1     988418
2     653187
3    1557977
4    1597518
Name: subways: total estimated ridership, dtype: int64


Compute average ridership per bucket

In [166]:
bucket_avg = df.groupby('rain_bucket')['subways: total estimated ridership'].mean().reset_index()
bucket_avg

/var/folders/06/km8cwz_s5vz3wc_c7f7rjxpr0000gn/T/ipykernel_21042/3582854271.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,rain_bucket,subways: total estimated ridership
0,No rain,2.159115e+06
1,Light,2.342732e+06
2,Moderate,2.369157e+06
3,Heavy,2.367627e+06


In [167]:
fig = px.scatter(
    bucket_avg,
    x = "rain_bucket",
    y= "subways: total estimated ridership",
    title = "Average Subway Ridership by Rainfall Level"
)
fig.update_layout(xaxis_title_text = "Rainfall bucket")
fig.update_layout(yaxis_title_text = "Avg daily ridership")
fig.show()
fig.write_image("project3_rain_bucket.png")


![](project3_rain_bucket.png)

# Conclusion

After merging NYC subway ridership data with Central Park precipitation records for 2021–2022, the analysis indicates that rainfall does influence subway ridership, but the effect is not uniform across all rain levels.
Ridership on dry days (“No rain”) is noticeably lower, likely because these days include weekends, holidays, and other low-demand periods. Once rainfall begins—even at light levels (0.01–0.1 inches)—average ridership actually increases, reflecting the fact that most light-rain days happen on weekdays when commuting volume is naturally higher.
The decline appears only at moderate (0.1–0.5 inches) and heavy (>0.5 inches) rainfall levels, where ridership slightly drops compared to light-rain days. However, the magnitude of this decline is relatively small, suggesting that rain affects rider comfort more than overall system usage.

Overall, the results provide partial support for the hypothesis:

Significant rain does reduce ridership, but only once rainfall is substantial, and even then, the decline is modest.